In [1]:

P = 2**127 - 1           
Q = P - 1              
G = 3
H = 5

import random
from typing import List, Dict

def pedersen_commit_scalar(m: int, r: int) -> int:
    return (pow(G, int(m), P) * pow(H, int(r), P)) % P

def pedersen_commit_vector(x: List[int], s: List[int]) -> List[int]:
    assert len(x) == len(s)
    return [pedersen_commit_scalar(x[i], s[i]) for i in range(len(x))]

def mod_q(x: int) -> int:
    return x % Q

def group_inv(a: int) -> int:
    return pow(a, P-2, P)


In [2]:
import math

def sample_cos_instance(ell: int,
                        max_entry: int,
                        P_bound: int,
                        alpha: int = 1):

    x = [random.randint(0, max_entry) for _ in range(ell)]
    y = [random.randint(0, max_entry) for _ in range(ell)]

    # 保证 y ≠ 0
    if all(v == 0 for v in y):
        y[0] = 1

    s = [random.randint(0, Q-1) for _ in range(ell)]
    C = pedersen_commit_vector(x, s)

    return x, s, y, alpha, P_bound, C

ELL = 4
P_bound = 50
x_cos, s_cos, y_cos, alpha_cos, P_bound, C_cos = sample_cos_instance(ELL, max_entry=10,
                                                                     P_bound=P_bound,
                                                                     alpha=1)

print("x =", x_cos)
print("y =", y_cos)
print("C =", C_cos)


x = [10, 10, 3, 5]
y = [3, 4, 4, 0]
C = [72420623244246773643408558857178576860, 113554276354485948558161772003068269138, 129222696588126269953528705861113231196, 42040550985588392470451696185878380730]


In [3]:
INDEX_MAIN = ["plus", "times", "beta", "y", "mu", "chi"]      # {+, ×, β, y, μ, χ}
INDEX_EXTRA = ["diamond", "four"]                             # {◊, 4}

def prover_first_message_cos(x, s, y, alpha: int, P_bound: int):
    ell = len(x)

    mu_vec  = [random.randint(0, Q-1) for _ in range(ell)]
    rho_vec = [random.randint(0, Q-1) for _ in range(ell)]
    T = pedersen_commit_vector(mu_vec, rho_vec)

    # β = α^2 ||y||_2^2
    y_norm_sq = sum(v*v for v in y)
    beta_param = alpha * alpha * y_norm_sq

    dot_x_mu = sum(x[i] * mu_vec[i] for i in range(ell))
    dot_mu_mu = sum(mu_vec[i] * mu_vec[i] for i in range(ell))
    dot_x_x = sum(x[i] * x[i] for i in range(ell))
    dot_x_y = sum(x[i] * y[i] for i in range(ell))
    dot_mu_y = sum(mu_vec[i] * y[i] for i in range(ell))

    x_vals: Dict[str, int] = {}
    x_vals["plus"]  = 2 * dot_x_mu                     # x_+ = 2<x, μ>
    x_vals["times"] = dot_mu_mu                        # x_× = <μ, μ>
    x_vals["beta"]  = beta_param * dot_x_x             # x_β = β<x, x>
    x_vals["y"]     = dot_x_y * dot_x_y               # x_y = <x, y>^2
    x_vals["mu"]    = dot_mu_y * dot_mu_y             # x_μ = <μ, y>^2
    x_vals["chi"]   = 2 * dot_x_y * dot_mu_y          # x_χ = 2<x, y><μ, y>

    # s_i for i ∈ {+, ×, β, y, μ, χ}
    s_vals: Dict[str, int] = {
        label: random.randint(0, Q-1) for label in INDEX_MAIN
    }

    mu_vals: Dict[str, int] = {}
    rho_vals: Dict[str, int] = {}


    for label in INDEX_MAIN + INDEX_EXTRA:
        rho_vals[label] = random.randint(0, Q-1)


    base_beta = random.randint(0, Q - P_bound - 1)
    delta = random.randint(0, P_bound - 1)
    mu_vals["beta"] = base_beta
    mu_vals["y"]    = base_beta + delta


    for label in ["plus", "times", "mu", "chi"]:
        mu_vals[label] = random.randint(0, Q-1)

    # μ_◊ = μ_4 = 0
    mu_vals["diamond"] = 0
    mu_vals["four"]    = 0


    c_dict: Dict[str, int] = {}
    for label in INDEX_MAIN:
        c_dict[label] = pedersen_commit_scalar(x_vals[label], s_vals[label])

    t_dict: Dict[str, int] = {}
    for label in INDEX_MAIN + INDEX_EXTRA:
        t_dict[label] = pedersen_commit_scalar(mu_vals[label], rho_vals[label])

    first_msg = {
        "mu_vec": mu_vec,
        "rho_vec": rho_vec,
        "T": T,

        "beta": beta_param,
        "x_vals": x_vals,
        "s_vals": s_vals,
        "mu_vals": mu_vals,
        "rho_vals": rho_vals,

        "c_dict": c_dict,
        "t_dict": t_dict,
    }
    return first_msg

first_msg_cos = prover_first_message_cos(x_cos, s_cos, y_cos, alpha_cos, P_bound)
print("first message keys =", list(first_msg_cos.keys()))


first message keys = ['mu_vec', 'rho_vec', 'T', 'beta', 'x_vals', 's_vals', 'mu_vals', 'rho_vals', 'c_dict', 't_dict']


In [4]:
def verifier_challenge_cos(P_bound: int) -> int:

    return random.randint(P_bound, Q-1)

c_cos = verifier_challenge_cos(P_bound)
print("challenge c =", c_cos)


challenge c = 26827401933587261901202717686335770329


In [5]:
def prover_response_cos(x, s, y, alpha: int, P_bound: int,
                        first_msg, c: int):
    ell = len(x)

    mu_vec  = first_msg["mu_vec"]
    rho_vec = first_msg["rho_vec"]
    beta_param = first_msg["beta"]
    x_vals  = first_msg["x_vals"]
    s_vals  = first_msg["s_vals"]
    mu_vals = first_msg["mu_vals"]
    rho_vals = first_msg["rho_vals"]


    Rx = [c * x[i] + mu_vec[i] for i in range(ell)]
    Rs = [mod_q(c * s[i] + rho_vec[i]) for i in range(ell)]


    s_beta = s_vals["beta"]
    s_plus = s_vals["plus"]
    s_times = s_vals["times"]

    s_y   = s_vals["y"]
    s_chi = s_vals["chi"]
    s_mu  = s_vals["mu"]

    s_diamond = mod_q(-(beta_param * (c*c * s_beta + c * s_plus + s_times)))
    s_four    = mod_q(-(c*c * s_y + 2 * c * s_chi + s_mu))


    Rx_vals: Dict[str, int] = {}
    Rs_vals: Dict[str, int] = {}

    for label in INDEX_MAIN:
        Rx_vals[label] = c * x_vals[label] + mu_vals[label]
        Rs_vals[label] = mod_q(c * s_vals[label] + rho_vals[label])

    Rs_diamond = mod_q(c * s_diamond + rho_vals["diamond"])
    Rs_four    = mod_q(c * s_four    + rho_vals["four"])

    response = {
        "Rx": Rx,
        "Rs": Rs,
        "Rx_vals": Rx_vals,
        "Rs_vals": Rs_vals,
        "Rs_diamond": Rs_diamond,
        "Rs_four": Rs_four,
        "s_diamond": s_diamond,
        "s_four": s_four,
    }
    return response

resp_cos = prover_response_cos(x_cos, s_cos, y_cos, alpha_cos, P_bound,
                               first_msg_cos, c_cos)

print("Rx (first 4) =", resp_cos["Rx"])
print("Rs (first 4) =", resp_cos["Rs"])


Rx (first 4) = [390274200285864735878752642479742295674, 399616827834469673806480586427149949507, 129864537422541801801080207061932885964, 178798110970323304229286516062740840579]
Rs (first 4) = [151819798505009783260236319306511624973, 50006285137996541489741467089336623585, 17664426182431551122099326752332650400, 33089706340957128503902850660169795314]


In [6]:
def verifier_check_cos(C, x, y, alpha: int, P_bound: int,
                       first_msg, response, c: int) -> bool:
    ell = len(C)

    T = first_msg["T"]
    beta_param = first_msg["beta"]
    c_dict = first_msg["c_dict"]
    t_dict = first_msg["t_dict"]

    Rx = response["Rx"]
    Rs = response["Rs"]
    Rx_vals = response["Rx_vals"]
    Rs_vals = response["Rs_vals"]
    Rs_diamond = response["Rs_diamond"]
    Rs_four = response["Rs_four"]


    inner_Rx = sum(v*v for v in Rx)
    num_dia = pow(G, beta_param * inner_Rx, P)

    den_dia = pow(c_dict["beta"],  beta_param * c * c, P)
    den_dia = (den_dia * pow(c_dict["plus"],  beta_param * c, P)) % P
    den_dia = (den_dia * pow(c_dict["times"], beta_param,      P)) % P
    c_diamond = (num_dia * group_inv(den_dia)) % P


    inner_Rx_y = sum(Rx[i] * y[i] for i in range(ell))
    num_four = pow(G, inner_Rx_y * inner_Rx_y, P)

    den_four = pow(c_dict["y"],   c * c, P)
    den_four = (den_four * pow(c_dict["chi"], c, P)) % P
    den_four = (den_four * c_dict["mu"]) % P
    c_four = (num_four * group_inv(den_four)) % P

    # 2) 标量等式
    for label in INDEX_MAIN:
        left  = (pow(G, Rx_vals[label], P) * pow(H, Rs_vals[label], P)) % P
        right = (t_dict[label] * pow(c_dict[label], c, P)) % P
        if left != right:
            print(f"[cos fail] scalar eq for {label}")
            return False


    for j in range(ell):
        left  = (pow(G, Rx[j], P) * pow(H, Rs[j], P)) % P
        right = (T[j] * pow(C[j], c, P)) % P
        if left != right:
            print(f"[cos fail] vector eq at coord {j}")
            return False


    left_dia  = pow(H, Rs_diamond, P)
    right_dia = (t_dict["diamond"] * pow(c_diamond, c, P)) % P
    if left_dia != right_dia:
        print("[cos fail] diamond eq")
        return False

    left_four  = pow(H, Rs_four, P)
    right_four = (t_dict["four"] * pow(c_four, c, P)) % P
    if left_four != right_four:
        print("[cos fail] four eq")
        return False

    # 5) R_xy - R_xβ ≤ (c+1) P_bound
    Rxy  = Rx_vals["y"]
    Rxb  = Rx_vals["beta"]
    if Rxy - Rxb > (c + 1) * P_bound:
        print("[cos fail] bound")
        return False

    return True

accepted_cos = verifier_check_cos(C_cos, x_cos, y_cos,
                                  alpha_cos, P_bound,
                                  first_msg_cos, resp_cos, c_cos)
print("Verifier accepts (cos)?", accepted_cos)


[cos fail] diamond eq
Verifier accepts (cos)? False
